# Data Environment

The OpenMP  _data environment_  contains data attributes of variables and objects.  Many constructs (such as __parallel__, __simd__, __task__)  accept clauses to control  _data-sharing_  attributes of referenced variables in the construct, where  _data-sharing_  applies to whether the attribute of the variable is  _shared_ ,  is  _private_  storage, or has special operational characteristics  (as found in the __firstprivate__, __lastprivate__, __linear__, or __reduction__ clause).

The data environment for a device (distinguished as a  _device data environment_ ) is controlled on the host by  _data-mapping_  attributes, which determine the relationship of the data on the host, the  _original_  data, and the data on the device, the  _corresponding_  data.

DATA-SHARING ATTRIBUTES

Data-sharing attributes of variables can be classified as being  _predetermined_ ,  _explicitly determined_  or  _implicitly determined_ .

Certain variables and objects have predetermined attributes.   A commonly found case is the loop iteration variable in associated loops  of a __for__ or __do__ construct. It has a private data-sharing attribute. Variables with predetermined data-sharing attributes cannot be listed in a data-sharing clause; but there are some exceptions (mainly concerning loop iteration variables).

Variables with explicitly determined data-sharing attributes are those that are referenced in a given construct and are listed in a data-sharing attribute clause on the construct. Some of the common data-sharing clauses are: __shared__, __private__, __firstprivate__, __lastprivate__,  __linear__, and __reduction__.

Variables with implicitly determined data-sharing attributes are those that are referenced in a given construct, do not have predetermined data-sharing attributes, and are not listed in a data-sharing attribute clause of an enclosing construct. For a complete list of variables and objects with predetermined and implicitly determined attributes, please refer to the  _Data-sharing Attribute Rules for Variables Referenced in a Construct_  subsection of the OpenMP Specifications document.

DATA-MAPPING ATTRIBUTES

The __map__ clause on a device construct explicitly specifies how the list items in the clause are mapped from the encountering task's data environment (on the host) to the corresponding item in the device data environment (on the device). The common  _list items_  are arrays, array sections, scalars, pointers, and structure elements (members).

Procedures and global variables have predetermined data mapping if they appear within the list or block of a __declare__ __target__ directive. Also, a C/C++ pointer is mapped as a zero-length array section, as is a C++ variable that is a reference to a pointer.

Without explicit mapping, non-scalar and non-pointer variables within the scope of the __target__ construct are implicitly mapped with a  _map-type_  of __tofrom__. Without explicit mapping, scalar variables within the scope of the __target__ construct are not mapped, but have an implicit firstprivate data-sharing attribute. (That is, the value of the original variable is given to a private variable of the same name on the device.) This behavior can be changed with the __defaultmap__ clause.

The __map__ clause can appear on __target__, __target data__ and  __target enter/exit data__ constructs.  The operations of creation and removal of device storage as well as assignment of the original list item  values to the corresponding list items may be complicated when the list  item appears on multiple constructs or when the host and device storage  is shared. In these cases the item's reference count, the number of times it has been referenced (+1 on entry and -1 on exited) in nested (structured) map regions and/or accumulative (unstructured) mappings, determines the operation. Details of the __map__ clause and reference count operation are specified  in the  _map Clause_  subsection of the OpenMP Specifications document.

## __threadprivate__ Directive

The following examples demonstrate how to use the __threadprivate__ directive   to give each thread a separate counter.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: threadprivate.1
* type: C
*/
int counter = 0;
#pragma omp threadprivate(counter)

int increment_counter()
{
  counter++;
  return(counter);
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: threadprivate.1
! type: F-fixed
      INTEGER FUNCTION INCREMENT_COUNTER()
        COMMON/INC_COMMON/COUNTER
!$OMP   THREADPRIVATE(/INC_COMMON/)

        COUNTER = COUNTER +1
        INCREMENT_COUNTER = COUNTER
        RETURN
      END FUNCTION INCREMENT_COUNTER

The following example uses __threadprivate__ on a static variable:

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: threadprivate.2
* type: C
*/
int increment_counter_2()
{
  static int counter = 0;
  #pragma omp threadprivate(counter)
  counter++;
  return(counter);
}

The following example demonstrates unspecified behavior for the initialization  of a __threadprivate__ variable. A __threadprivate__  variable is initialized  once at an unspecified point before its first reference. Because __a__ is  constructed using the value of __x__  (which is modified by the statement  __x++__), the value of __a.val__  at the start of the __parallel__  region could be either 1 or 2. This problem is avoided for __b__, which uses  an auxiliary __const__ variable and a copy-constructor.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: threadprivate.3
* type: C++
*/
class T {
  public:
    int val;
    T (int);
    T (const T&);
};

T :: T (int v){
   val = v;
}

T :: T (const T& t) {
   val = t.val;
}

void g(T a, T b){
   a.val += b.val;
}

int x = 1;
T a(x);
const T b_aux(x); /* Capture value of x = 1 */
T b(b_aux);
#pragma omp threadprivate(a, b)

void f(int n) {
   x++;
   #pragma omp parallel for
   /* In each thread:
    * a is constructed from x (with value 1 or 2?)
    * b is copy-constructed from b_aux
    */

   for (int i=0; i<n; i++) {
       g(a, b); /* Value of a is unspecified. */
   }
}

The following examples show non-conforming uses and correct uses of the __threadprivate__  directive.

The following example is non-conforming because the common block is not declared  local to the subroutine that refers to it:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: threadprivate.2
! type: F-fixed
      MODULE INC_MODULE
        COMMON /T/ A
      END MODULE INC_MODULE

      SUBROUTINE INC_MODULE_WRONG()
        USE INC_MODULE
!$OMP   THREADPRIVATE(/T/)
      !non-conforming because /T/ not declared in INC_MODULE_WRONG
      END SUBROUTINE INC_MODULE_WRONG

The following example is also non-conforming because the common block is not declared  local to the subroutine that refers to it:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: threadprivate.3
! type: F-fixed
      SUBROUTINE INC_WRONG()
        COMMON /T/ A
!$OMP   THREADPRIVATE(/T/)

        CONTAINS
          SUBROUTINE INC_WRONG_SUB()
!$OMP       PARALLEL COPYIN(/T/)
      !non-conforming because /T/ not declared in INC_WRONG_SUB
!$OMP       END PARALLEL
          END SUBROUTINE INC_WRONG_SUB
      END SUBROUTINE INC_WRONG

The following example is a correct rewrite of the previous example:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: threadprivate.4
! type: F-fixed
       SUBROUTINE INC_GOOD()
        COMMON /T/ A
!$OMP   THREADPRIVATE(/T/)

        CONTAINS
          SUBROUTINE INC_GOOD_SUB()
            COMMON /T/ A
!$OMP       THREADPRIVATE(/T/)

!$OMP       PARALLEL COPYIN(/T/)
!$OMP       END PARALLEL
         END SUBROUTINE INC_GOOD_SUB
       END SUBROUTINE INC_GOOD

The following is an example of the use of __threadprivate__ for local variables:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: threadprivate.5
! type: F-fixed
      PROGRAM INC_GOOD2
        INTEGER, ALLOCATABLE, SAVE :: A(:)
        INTEGER, POINTER, SAVE :: PTR
        INTEGER, SAVE :: I
        INTEGER, TARGET :: TARG
        LOGICAL :: FIRSTIN = .TRUE.
!$OMP   THREADPRIVATE(A, I, PTR)

        ALLOCATE (A(3))
        A = (/1,2,3/)
        PTR => TARG
        I = 5

!$OMP   PARALLEL COPYIN(I, PTR)
!$OMP     CRITICAL
            IF (FIRSTIN) THEN
              TARG = 4           ! Update target of ptr
              I = I + 10
              IF (ALLOCATED(A)) A = A + 10
              FIRSTIN = .FALSE.
            END IF

            IF (ALLOCATED(A)) THEN
              PRINT *, 'a = ', A
            ELSE
              PRINT *, 'A is not allocated'
            END IF

            PRINT *, 'ptr = ', PTR
            PRINT *, 'i = ', I
            PRINT *

!$OMP     END CRITICAL
!$OMP   END PARALLEL
      END PROGRAM INC_GOOD2

The above program, if executed by two threads, will print one of the following  two sets of output:

__a = 11 12 13__  
__ptr = 4__  __i = 15__

__A is not allocated__  
__ptr = 4__  
__i = 5__

or

__A is not allocated__  
__ptr = 4__  
__i = 15__

__a = 1 2 3__  
__ptr = 4__  
__i = 5__

The following is an example of the use of __threadprivate__ for module variables:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: threadprivate.6
! type: F-fixed
      MODULE INC_MODULE_GOOD3
        REAL, POINTER :: WORK(:)
        SAVE WORK
!$OMP   THREADPRIVATE(WORK)
      END MODULE INC_MODULE_GOOD3

      SUBROUTINE SUB1(N)
      USE INC_MODULE_GOOD3
!$OMP   PARALLEL PRIVATE(THE_SUM)
        ALLOCATE(WORK(N))
        CALL SUB2(THE_SUM)
       WRITE(*,*)THE_SUM
!$OMP   END PARALLEL
      END SUBROUTINE SUB1

      SUBROUTINE SUB2(THE_SUM)
        USE INC_MODULE_GOOD3
        WORK(:) = 10
        THE_SUM=SUM(WORK)
      END SUBROUTINE SUB2

      PROGRAM INC_GOOD3
        N = 10
        CALL SUB1(N)
      END PROGRAM INC_GOOD3

The following example illustrates initialization of __threadprivate__ variables  for class-type __T__. __t1__ is default constructed, __t2__ is constructed  taking a constructor accepting one argument of integer type, __t3__ is copy  constructed with argument __f()__:

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: threadprivate.4
* type: C++
*/
struct T { T (); T (int); ~T (); int t; };
int f();
static T t1;
#pragma omp threadprivate(t1)
static T t2( 23 );
#pragma omp threadprivate(t2)
static T t3 = f();
#pragma omp threadprivate(t3)

The following example illustrates the use of __threadprivate__ for static  class members. The __threadprivate__ directive for a static class member must  be placed inside the class definition.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: threadprivate.5
* type: C++
*/
class T {
 public:
  static int i;
#pragma omp threadprivate(i)
};

## __default(none)__ Clause

The following example distinguishes the variables that are affected by the __default(none)__  clause from those that are not.

Beginning with OpenMP 4.0, variables with __const__-qualified type and no mutable member  are no longer predetermined shared.  Thus, these variables (variable  _c_  in the example)  need to be explicitly listed in data-sharing attribute clauses when the __default(none)__ clause is specified.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: default_none.1
* type: C
*/
#include <omp.h>
int x, y, z[1000];
#pragma omp threadprivate(x)

void default_none(int a) {
  const int c = 1;
  int i = 0;

  #pragma omp parallel default(none) private(a) shared(z, c)
  {
     int j = omp_get_num_threads();
          /* O.K.  - j is declared within parallel region */
     a = z[j];   /* O.K.  - a is listed in private clause */
                 /*       - z is listed in shared clause */
     x = c;      /* O.K.  - x is threadprivate */
                 /*       - c has const-qualified type and
                              is listed in shared clause */
     z[i] = y;   /* Error - cannot reference i or y here */

  #pragma omp for firstprivate(y)
         /* Error - Cannot reference y in the firstprivate clause */
     for (i=0; i<10 ; i++) {
        z[i] = i; /* O.K. - i is the loop iteration variable */
     }

     z[i] = y;   /* Error - cannot reference i or y here */
  }
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: default_none.1
! type: F-fixed
      SUBROUTINE DEFAULT_NONE(A)
      INCLUDE "omp_lib.h"     ! or USE OMP_LIB

      INTEGER A

      INTEGER X, Y, Z(1000)
      COMMON/BLOCKX/X
      COMMON/BLOCKY/Y
      COMMON/BLOCKZ/Z
!$OMP THREADPRIVATE(/BLOCKX/)

        INTEGER I, J
        i = 1

!$OMP   PARALLEL DEFAULT(NONE) PRIVATE(A) SHARED(Z) PRIVATE(J)
          J = OMP_GET_NUM_THREADS();
                   ! O.K.  - J is listed in PRIVATE clause
          A = Z(J) ! O.K.  - A is listed in PRIVATE clause
                   !       - Z is listed in SHARED clause
          X = 1    ! O.K.  - X is THREADPRIVATE
          Z(I) = Y ! Error - cannot reference I or Y here

!$OMP DO firstprivate(y)
    ! Error - Cannot reference y in the firstprivate clause
          DO I = 1,10
             Z(I) = I ! O.K. - I is the loop iteration variable
          END DO


          Z(I) = Y    ! Error - cannot reference I or Y here
!$OMP   END PARALLEL
      END SUBROUTINE DEFAULT_NONE

## __private__ Clause

In the following example, the values of original list items  _i_  and  _j_   are retained on exit from the __parallel__ region, while the private list  items  _i_  and  _j_  are modified within the __parallel__ construct.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: private.1
* type: C
*/
#include <stdio.h>
#include <assert.h>

int main()
{
  int i, j;
  int *ptr_i, *ptr_j;

  i = 1;
  j = 2;

  ptr_i = &i;
  ptr_j = &j;

  #pragma omp parallel private(i) firstprivate(j)
  {
    i = 3;
    j = j + 2;
    assert (*ptr_i == 1 && *ptr_j == 2);
  }

  assert(i == 1 && j == 2);

  return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: private.1
! type: F-fixed
      PROGRAM PRIV_EXAMPLE
        INTEGER I, J

        I = 1
        J = 2

!$OMP   PARALLEL PRIVATE(I) FIRSTPRIVATE(J)
          I = 3
          J = J + 2
!$OMP   END PARALLEL

        PRINT *, I, J  ! I .eq. 1 .and. J .eq. 2
      END PROGRAM PRIV_EXAMPLE

In the following example, all uses of the variable  _a_  within the loop construct  in the routine  _f_  refer to a private list item  _a_ , while it is  unspecified whether references to  _a_  in the routine  _g_  are to a  private list item or the original list item.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: private.2
* type: C
*/
int a;

void g(int k) {
  a = k; /* Accessed in the region but outside of the construct;
          * therefore unspecified whether original or private list
          * item is modified. */
}


void f(int n) {
  int a = 0;

  #pragma omp parallel for private(a)
   for (int i=1; i<n; i++) {
       a = i;
       g(a*2);     /* Private copy of "a" */
    }
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: private.2
! type: F-fixed
      MODULE PRIV_EXAMPLE2
        REAL A

        CONTAINS

          SUBROUTINE G(K)
            REAL K
            A = K  ! Accessed in the region but outside of the
                   ! construct; therefore unspecified whether
                   ! original or private list item is modified.
          END SUBROUTINE G

          SUBROUTINE F(N)
          INTEGER N
          REAL A

            INTEGER I
!$OMP       PARALLEL DO PRIVATE(A)
              DO I = 1,N
                A = I
                CALL G(A*2)
              ENDDO
!$OMP       END PARALLEL DO
          END SUBROUTINE F

      END MODULE PRIV_EXAMPLE2

The following example demonstrates that a list item that appears in a __private__   clause in a __parallel__ construct may also appear in a __private__   clause in an enclosed worksharing construct, which results in an additional private  copy.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: private.3
* type: C
*/
#include <assert.h>
void priv_example3()
{
  int i, a;

  #pragma omp parallel private(a)
  {
     a = 1;
    #pragma omp parallel for private(a)
      for (i=0; i<10; i++)
     {
       a = 2;
     }
    assert(a == 1);
  }
}

In [ ]:
!%compiler: gfortran
!!%cflags: -fopenmp

! name: private.3
! type: F-fixed
      SUBROUTINE PRIV_EXAMPLE3()
        INTEGER I, A

!$OMP   PARALLEL PRIVATE(A)
         A = 1
!$OMP     PARALLEL DO PRIVATE(A)
          DO I = 1, 10
            A = 2
          END DO
!$OMP     END PARALLEL DO
        PRINT *, A ! Outer A still has value 1
!$OMP   END PARALLEL
      END SUBROUTINE PRIV_EXAMPLE3

## Fortran Private Loop Iteration Variables

In general loop iteration variables will be private, when used in the  _do-loop_   of a __do__ and __parallel do__ construct or in sequential loops in a  __parallel__ construct (see Section 2.7.1 and Section 2.14.1 of  the OpenMP 4.0 specification). In the following example of a sequential  loop in a __parallel__ construct the loop iteration variable  _I_  will  be private.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_loopvar.1
! type: F-free
SUBROUTINE PLOOP_1(A,N)
INCLUDE "omp_lib.h"      ! or USE OMP_LIB

REAL A(*)
INTEGER I, MYOFFSET, N

!$OMP PARALLEL PRIVATE(MYOFFSET)
       MYOFFSET = OMP_GET_THREAD_NUM()*N
       DO I = 1, N
         A(MYOFFSET+I) = FLOAT(I)
       ENDDO
!$OMP END PARALLEL

END SUBROUTINE PLOOP_1

In exceptional cases, loop iteration variables can be made shared, as in the following  example:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_loopvar.2
! type: F-free
SUBROUTINE PLOOP_2(A,B,N,I1,I2)
REAL A(*), B(*)
INTEGER I1, I2, N

!$OMP PARALLEL SHARED(A,B,I1,I2)
!$OMP SECTIONS
!$OMP SECTION
     DO I1 = I1, N
       IF (A(I1).NE.0.0) EXIT
     ENDDO
!$OMP SECTION
     DO I2 = I2, N
       IF (B(I2).NE.0.0) EXIT
     ENDDO
!$OMP END SECTIONS
!$OMP SINGLE
    IF (I1.LE.N) PRINT *, 'ITEMS IN A UP TO ', I1, 'ARE ALL ZERO.'
    IF (I2.LE.N) PRINT *, 'ITEMS IN B UP TO ', I2, 'ARE ALL ZERO.'
!$OMP END SINGLE
!$OMP END PARALLEL

END SUBROUTINE PLOOP_2

Note however that the use of shared loop iteration variables can easily lead to  race conditions.

## Fortran Restrictions on __shared__ and __private__ Clauses with Common Blocks

When a named common block is specified in a __private__, __firstprivate__,  or __lastprivate__ clause of a construct, none of its members may be declared  in another data-sharing attribute clause on that construct. The following examples  illustrate this point.

The following example is conforming:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sp_common.1
! type: F-fixed
      SUBROUTINE COMMON_GOOD()
        COMMON /C/ X,Y
        REAL X, Y

!$OMP   PARALLEL PRIVATE (/C/)
          ! do work here
!$OMP   END PARALLEL
!$OMP   PARALLEL SHARED (X,Y)
          ! do work here
!$OMP   END PARALLEL
      END SUBROUTINE COMMON_GOOD

The following example is also conforming:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sp_common.2
! type: F-fixed
      SUBROUTINE COMMON_GOOD2()
        COMMON /C/ X,Y
        REAL X, Y
        INTEGER I
!$OMP   PARALLEL
!$OMP     DO PRIVATE(/C/)
          DO I=1,1000
            ! do work here
          ENDDO
!$OMP     END DO
!$OMP     DO PRIVATE(X)
          DO I=1,1000
            ! do work here
          ENDDO
!$OMP     END DO
!$OMP   END PARALLEL
      END SUBROUTINE COMMON_GOOD2

The following example is conforming:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sp_common.3
! type: F-fixed
      SUBROUTINE COMMON_GOOD3()
        COMMON /C/ X,Y
!$OMP   PARALLEL PRIVATE (/C/)
          ! do work here
!$OMP   END PARALLEL
!$OMP   PARALLEL SHARED (/C/)
          ! do work here
!$OMP   END PARALLEL
      END SUBROUTINE COMMON_GOOD3

The following example is non-conforming because __x__ is a constituent element  of __c__:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sp_common.4
! type: F-fixed
      SUBROUTINE COMMON_WRONG()
        COMMON /C/ X,Y
! Incorrect because X is a constituent element of C
!$OMP   PARALLEL PRIVATE(/C/), SHARED(X)
          ! do work here
!$OMP   END PARALLEL
      END SUBROUTINE COMMON_WRONG

The following example is non-conforming because a common block may not be declared  both shared and private:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sp_common.5
! type: F-fixed
      SUBROUTINE COMMON_WRONG2()
        COMMON /C/ X,Y
! Incorrect: common block C cannot be declared both
! shared and private
!$OMP   PARALLEL PRIVATE (/C/), SHARED(/C/)
          ! do work here
!$OMP   END PARALLEL

      END SUBROUTINE COMMON_WRONG2

## Fortran Restrictions on Storage Association with the __private__ Clause

The following non-conforming examples illustrate the implications of the __private__  clause rules with regard to storage association.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sa_private.1
! type: F-fixed
       SUBROUTINE SUB()
       COMMON /BLOCK/ X
       PRINT *,X             ! X is undefined
       END SUBROUTINE SUB

       PROGRAM PRIV_RESTRICT
         COMMON /BLOCK/ X
         X = 1.0
!$OMP    PARALLEL PRIVATE (X)
         X = 2.0
         CALL SUB()
!$OMP    END PARALLEL
      END PROGRAM PRIV_RESTRICT

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sa_private.2
! type: F-fixed
      PROGRAM PRIV_RESTRICT2
        COMMON /BLOCK2/ X
        X = 1.0

!$OMP   PARALLEL PRIVATE (X)
          X = 2.0
          CALL SUB()
!$OMP   END PARALLEL

       CONTAINS

          SUBROUTINE SUB()
          COMMON /BLOCK2/ Y

          PRINT *,X               ! X is undefined
          PRINT *,Y               ! Y is undefined
          END SUBROUTINE SUB

       END PROGRAM PRIV_RESTRICT2

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sa_private.3
! type: F-fixed
      PROGRAM PRIV_RESTRICT3
        EQUIVALENCE (X,Y)
        X = 1.0

!$OMP   PARALLEL PRIVATE(X)
          PRINT *,Y                  ! Y is undefined
          Y = 10
          PRINT *,X                  ! X is undefined
!$OMP   END PARALLEL
      END PROGRAM PRIV_RESTRICT3

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sa_private.4
! type: F-fixed
      PROGRAM PRIV_RESTRICT4
        INTEGER I, J
        INTEGER A(100), B(100)
        EQUIVALENCE (A(51), B(1))

!$OMP PARALLEL DO DEFAULT(PRIVATE) PRIVATE(I,J) LASTPRIVATE(A)
          DO I=1,100
             DO J=1,100
               B(J) = J - 1
             ENDDO

             DO J=1,100
               A(J) = J   ! B becomes undefined at this point
             ENDDO

             DO J=1,50
               B(J) = B(J) + 1  ! B is undefined
                         ! A becomes undefined at this point
             ENDDO
          ENDDO
!$OMP END PARALLEL DO       ! The LASTPRIVATE write for A has
                            ! undefined results

         PRINT *, B    ! B is undefined since the LASTPRIVATE
                       ! write of A was not defined
       END PROGRAM PRIV_RESTRICT4

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: fort_sa_private.5
! type: F-fixed
! version:    omp_5.1
      SUBROUTINE SUB1(X)
        DIMENSION X(10)

        ! This use of X does not conform to the
        ! specification. It would be legal Fortran 90,
        ! but the OpenMP private directive allows the
        ! compiler to break the sequence association that
        ! A had with the rest of the common block.

        FORALL (I = 1:10) X(I) = I
      END SUBROUTINE SUB1

      PROGRAM PRIV_RESTRICT5
        COMMON /BLOCK5/ A

        DIMENSION B(10)
        EQUIVALENCE (A,B(1))

        ! the common block has to be at least 10 words
        A = 0

!$OMP   PARALLEL PRIVATE(/BLOCK5/)

          ! Without the private clause,
          ! we would be passing a member of a sequence
          ! that is at least ten elements long.
          ! With the private clause, A may no longer be
          ! sequence-associated.

          CALL SUB1(A)
!$OMP     MASKED
            PRINT *, A
!$OMP     END MASKED

!$OMP   END PARALLEL

## C/C++ Arrays in a __firstprivate__ Clause

The following example illustrates the size and value of list items of array or  pointer type in a __firstprivate__ clause . The size of new list items is  based on the type of the corresponding original list item, as determined by the  base language.

In this example:

*  The type of __A__ is array of two arrays of two ints.
*   The type of __B__ is adjusted to pointer to array of __n__  ints, because it is a function parameter.
*   The type of __C__ is adjusted to pointer to int, because  it is a function parameter.
*   The type of __D__ is array of two arrays of two ints.
*   The type of __E__ is array of __n__ arrays of __n__  ints.

Note that  __B__ and __E__ involve variable length array types.

The new items of array type are initialized as if each integer element of the original  array is assigned to the corresponding element of the new array. Those of pointer  type are initialized as if by assignment from the original item to the new item.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: carrays_fpriv.1
* type: C
*/
#include <assert.h>

int A[2][2] = {1, 2, 3, 4};

void f(int n, int B[n][n], int C[])
{
  int D[2][2] = {1, 2, 3, 4};
  int E[n][n];

  assert(n >= 2);
  E[1][1] = 4;

  #pragma omp parallel firstprivate(B, C, D, E)
  {
    assert(sizeof(B) == sizeof(int (*)[n]));
    assert(sizeof(C) == sizeof(int*));
    assert(sizeof(D) == 4 * sizeof(int));
    assert(sizeof(E) == n * n * sizeof(int));

    /* Private B and C have values of original B and C. */
    assert(&B[1][1] == &A[1][1]);
    assert(&C[3] == &A[1][1]);
    assert(D[1][1] == 4);
    assert(E[1][1] == 4);
  }
}

int main() {
  f(2, A, A[0]);
  return 0;
}

## __lastprivate__ Clause

Correct execution sometimes depends on the value that the last iteration of a loop  assigns to a variable. Such programs must list all such variables in a __lastprivate__  clause  so that the values of the variables are the same as when the loop is executed  sequentially.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: lastprivate.1
* type: C
*/
void lastpriv (int n, float *a, float *b)
{
  int i;

  #pragma omp parallel
  {
    #pragma omp for lastprivate(i)
    for (i=0; i<n-1; i++)
      a[i] = b[i] + b[i+1];
  }

  a[i]=b[i];      /* i == n-1 here */
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: lastprivate.1
! type: F-fixed
      SUBROUTINE LASTPRIV(N, A, B)

        INTEGER N
        REAL A(*), B(*)
        INTEGER I
!$OMP PARALLEL
!$OMP DO LASTPRIVATE(I)

        DO I=1,N-1
          A(I) = B(I) + B(I+1)
        ENDDO

!$OMP END PARALLEL
        A(I) = B(I)      ! I has the value of N here

      END SUBROUTINE LASTPRIV

The next example illustrates the use of the __conditional__ modifier in a __lastprivate__ clause to return the last value when it may not come from the last iteration of a loop. That is, users can preserve the serial equivalence semantics of the loop. The conditional lastprivate ensures the final value of the variable after the loop  is as if the loop iterations were executed in a sequential order.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: lastprivate.2
* type: C
* version: omp_5.0
*/
#include <math.h>

float condlastprivate(float *a, int n)
{
   float x = 0.0f;

   #pragma omp parallel for simd lastprivate(conditional: x)
   for (int k = 0; k < n; k++) {
      if (a[k] < 108.5 || a[k] > 208.5) {
         x = sinf(a[k]);
      }
   }

   return x;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: lastprivate.2
! type: F-free
! version: omp_5.0
function condlastprivate(a, n) result(x)
   implicit none
   real a(*), x
   integer n, k

   x = 0.0

   !$omp parallel do simd lastprivate(conditional: x)
   do k = 1, n
      if (a(k) < 108.5 .or. a(k) > 208.5) then
         x = sin(a(k))
      endif
   end do

end function condlastprivate

## Reduction

This section covers ways to perform reductions in parallel, task, taskloop, and SIMD regions.

### __reduction__ Clause

The following example demonstrates the __reduction__ clause; note that some  reductions can be expressed in the loop in several ways, as shown for the __max__  and __min__ reductions below:

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: reduction.1
* type: C
* version: omp_3.1
*/
#include <math.h>
void reduction1(float *x, int *y, int n)
{
  int i, b, c;
  float a, d;
  a = 0.0;
  b = 0;
  c = y[0];
  d = x[0];
  #pragma omp parallel for private(i) shared(x, y, n) \
                          reduction(+:a) reduction(^:b) \
                          reduction(min:c) reduction(max:d)
    for (i=0; i<n; i++) {
      a += x[i];
      b ^= y[i];
      if (c > y[i]) c = y[i];
      d = fmaxf(d,x[i]);
    }
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: reduction.1
! type: F-free
SUBROUTINE REDUCTION1(A, B, C, D, X, Y, N)
    REAL :: X(*), A, D
    INTEGER :: Y(*), N, B, C
    INTEGER :: I
    A = 0
    B = 0
    C = Y(1)
    D = X(1)
    !$OMP PARALLEL DO PRIVATE(I) SHARED(X, Y, N) REDUCTION(+:A) &
    !$OMP& REDUCTION(IEOR:B) REDUCTION(MIN:C)  REDUCTION(MAX:D)
      DO I=1,N
        A = A + X(I)
        B = IEOR(B, Y(I))
        C = MIN(C, Y(I))
        IF (D < X(I)) D = X(I)
      END DO

END SUBROUTINE REDUCTION1

A common implementation of the preceding example is to treat it as if it had been  written as follows:

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: reduction.2
* type: C
*/
#include <limits.h>
#include <math.h>
void reduction2(float *x, int *y, int n)
{
  int i, b, b_p, c, c_p;
  float a, a_p, d, d_p;
  a = 0.0f;
  b = 0;
  c = y[0];
  d = x[0];
  #pragma omp parallel shared(a, b, c, d, x, y, n) \
                          private(a_p, b_p, c_p, d_p)
  {
    a_p = 0.0f;
    b_p = 0;
    c_p = INT_MAX;
    d_p = -HUGE_VALF;
    #pragma omp for private(i)
    for (i=0; i<n; i++) {
      a_p += x[i];
      b_p ^= y[i];
      if (c_p > y[i]) c_p = y[i];
      d_p = fmaxf(d_p,x[i]);
    }
    #pragma omp critical
    {
      a += a_p;
      b ^= b_p;
      if( c > c_p ) c = c_p;
      d = fmaxf(d,d_p);
    }
  }
}

In [ ]:

!!%compiler: gfortran
!!%cflags: -fopenmp

! name: reduction.2
! type: F-free
  SUBROUTINE REDUCTION2(A, B, C, D, X, Y, N)
    REAL :: X(*), A, D
    INTEGER :: Y(*), N, B, C
    REAL :: A_P, D_P
    INTEGER :: I, B_P, C_P
    A = 0
    B = 0
    C = Y(1)
    D = X(1)
    !$OMP PARALLEL SHARED(X, Y, A, B, C, D, N) &
    !$OMP&         PRIVATE(A_P, B_P, C_P, D_P)
      A_P = 0.0
      B_P = 0
      C_P = HUGE(C_P)
      D_P = -HUGE(D_P)
      !$OMP DO PRIVATE(I)
      DO I=1,N
        A_P = A_P + X(I)
        B_P = IEOR(B_P, Y(I))
        C_P = MIN(C_P, Y(I))
        IF (D_P < X(I)) D_P = X(I)
      END DO
      !$OMP CRITICAL
        A = A + A_P
        B = IEOR(B, B_P)
        C = MIN(C, C_P)
        D = MAX(D, D_P)
      !$OMP END CRITICAL
    !$OMP END PARALLEL
  END SUBROUTINE REDUCTION2

The following program is non-conforming because the reduction is on the  **intrinsic procedure name** __MAX__ but that name has been redefined to be the variable  named __MAX__.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: reduction.3
! type: F-free
 PROGRAM REDUCTION_WRONG
 MAX = HUGE(0)
 M = 0

 !$OMP PARALLEL DO REDUCTION(MAX: M)
! MAX is no longer the intrinsic so this is non-conforming
 DO I = 1, 100
    CALL SUB(M,I)
 END DO

 END PROGRAM REDUCTION_WRONG

 SUBROUTINE SUB(M,I)
    M = MAX(M,I)
 END SUBROUTINE SUB

The following conforming program performs the reduction using the  **intrinsic procedure name** __MAX__ even though the intrinsic __MAX__ has been renamed  to __REN__.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: reduction.4
! type: F-free
MODULE M
   INTRINSIC MAX
END MODULE M

PROGRAM REDUCTION3
   USE M, REN => MAX
   N = 0
!$OMP PARALLEL DO REDUCTION(REN: N)     ! still does MAX
   DO I = 1, 100
      N = MAX(N,I)
   END DO
END PROGRAM REDUCTION3

The following conforming program performs the reduction using   _intrinsic procedure name_  __MAX__ even though the intrinsic __MAX__ has been renamed  to __MIN__.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: reduction.5
! type: F-free
MODULE MOD
   INTRINSIC MAX, MIN
END MODULE MOD

PROGRAM REDUCTION4
   USE MOD, MIN=>MAX, MAX=>MIN
   REAL :: R
   R = -HUGE(0.0)

!$OMP PARALLEL DO REDUCTION(MIN: R)     ! still does MAX
   DO I = 1, 1000
      R = MIN(R, SIN(REAL(I)))
   END DO
   PRINT *, R
END PROGRAM REDUCTION4

The following example is non-conforming because the initialization (__a =  0__) of the original list item __a__ is not synchronized with the update of  __a__ as a result of the reduction computation in the __for__ loop. Therefore,  the example may print an incorrect value for __a__.

To avoid this problem, the initialization of the original list item __a__  should complete before any update of __a__ as a result of the __reduction__  clause. This can be achieved by adding an explicit barrier after the assignment  __a = 0__, or by enclosing the assignment __a = 0__ in a __single__  directive (which has an implied barrier), or by initializing __a__ before  the start of the __parallel__ region.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: reduction.6
* type: C
* version:    omp_5.1
*/
#include <stdio.h>

int main (void)
{
  int a, i;

  #pragma omp parallel shared(a) private(i)
  {
    #pragma omp masked
    a = 0;

    // To avoid race conditions, add a barrier here.

    #pragma omp for reduction(+:a)
    for (i = 0; i < 10; i++) {
        a += i;
    }

    #pragma omp single
    printf ("Sum is %d\n", a);
  }
  return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: reduction.6
! type: F-fixed
! version:    omp_5.1
      INTEGER A, I

!$OMP PARALLEL SHARED(A) PRIVATE(I)

!$OMP MASKED
      A = 0
!$OMP END MASKED

      ! To avoid race conditions, add a barrier here.

!$OMP DO REDUCTION(+:A)
      DO I= 0, 9
         A = A + I
      END DO

!$OMP SINGLE
      PRINT *, "Sum is ", A
!$OMP END SINGLE

!$OMP END PARALLEL

      END

The following example demonstrates the reduction of array  _a_ .  In C/C++ this is illustrated by the explicit use of an array section  _a[0:N]_  in the __reduction__ clause.  The corresponding Fortran example uses array syntax supported in the base language.  As of the OpenMP 4.5 specification the explicit use of array section in the __reduction__ clause in Fortran is not permitted.  But this oversight has been fixed in the OpenMP 5.0 specification.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: reduction.7
* type: C
* version: omp_4.5
*/
#include <stdio.h>

#define N 100
void init(int n, float (*b)[N]);

int main(){

  int i,j;
  float a[N], b[N][N];

  init(N,b);

  for(i=0; i<N; i++) a[i]=0.0e0;

  #pragma omp parallel for reduction(+:a[0:N]) private(j)
  for(i=0; i<N; i++){
    for(j=0; j<N; j++){
       a[j] +=  b[i][j];
    }
  }
  printf(" a[0] a[N-1]: %f %f\n", a[0], a[N-1]);

  return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: reduction.7
! type: F-free
program array_red

  integer,parameter :: n=100
  integer           :: j
  real              :: a(n), b(n,n)

  call init(n,b)

  a(:) = 0.0e0

  !$omp parallel do reduction(+:a)
  do j = 1, n
     a(:) = a(:) + b(:,j)
  end do

  print*, " a(1) a(n): ", a(1), a(n)

end program

### Task Reduction

In OpenMP 5.0 the __task_reduction__ clause was created for the __taskgroup__ construct,  to allow reductions among explicit tasks that have an __in_reduction__ clause.

In the  _task_reduction.1_  example below a reduction is performed as the algorithm traverses a linked list. The reduction statement is assigned to be an explicit task using a __task__ construct and is specified to be a reduction participant with  the __in_reduction__ clause. A __taskgroup__ construct encloses the tasks participating in the reduction, and specifies, with the __task_reduction__ clause, that the taskgroup has tasks participating in a reduction.  After the __taskgroup__ region the original variable will contain  the final value of the reduction.

Note: The  _res_  variable is private in the  _linked_list_sum_  routine and is not required to be shared (as in the case of a __parallel__ construct reduction).

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       task_reduction.1
* type:       C
*/

#include<stdlib.h>
#include<stdio.h>
#define N 10

typedef struct node_tag {
    int val;
    struct node_tag *next;
} node_t;

int linked_list_sum(node_t *p)
{
    int res = 0;

    #pragma omp taskgroup task_reduction(+: res)
    {
        node_t* aux = p;
        while(aux != 0)
        {
            #pragma omp task in_reduction(+: res)
            res += aux->val;

            aux = aux->next;
        }
    }
    return res;
}


int main(int argc, char *argv[]) {
    int i;
//                           Create the root node.
    node_t* root = (node_t*) malloc(sizeof(node_t));
    root->val = 1;

    node_t* aux = root;

//                           Create N-1 more nodes.
    for(i=2;i<=N;++i){
        aux->next = (node_t*) malloc(sizeof(node_t));
        aux = aux->next;
        aux->val = i;
    }

    aux->next = 0;

    #pragma omp parallel
    #pragma omp single
    {
        int result = linked_list_sum(root);
        printf( "Calculated: %d  Analytic:%d\n", result, (N*(N+1)/2) );
    }

    return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       task_reduction.1
! type:       F-free

module m
    type node_t
        integer :: val
        type(node_t), pointer :: next
    end type
end module m

function linked_list_sum(p) result(res)
    use m
    implicit none
    type(node_t), pointer :: p
    type(node_t), pointer :: aux
    integer :: res

    res = 0

    !$omp taskgroup task_reduction(+: res)
        aux => p
        do while (associated(aux))
            !$omp task in_reduction(+: res)
                res = res + aux%val
            !$omp end task
            aux => aux%next
        end do
    !$omp end taskgroup
end function linked_list_sum


program main
    use m
    implicit none
    type(node_t), pointer :: root, aux
    integer :: res, i
    integer, parameter :: N=10

    interface
        function linked_list_sum(p) result(res)
            use m
            implicit none
            type(node_t), pointer :: p
            integer :: res
        end function
    end interface
!                       Create the root node.
    allocate(root)
    root%val = 1
    aux => root

!                       Create N-1 more nodes.
    do i = 2,N
        allocate(aux%next)
        aux => aux%next
        aux%val = i
    end do

    aux%next => null()

    !$omp parallel
    !$omp single
        res = linked_list_sum(root)
        print *, "Calculated:", res, " Analytic:", (N*(N+1))/2
    !$omp end single
    !$omp end parallel

end program main

In OpenMP 5.0 the __task__  _reduction-modifier_  for the __reduction__ clause was introduced to provide a means of performing reductions among implicit and explicit tasks.

The __reduction__ clause of a __parallel__ or worksharing construct may specify the __task__  _reduction-modifier_  to include explicit task reductions within their region, provided the reduction operators ( _reduction-identifiers_ ) and variables ( _list items_ ) of the participating tasks match those of the implicit tasks.

There are 2 reduction use cases (identified by USE CASE #) in the  _task_reduction.2_  example below.

In USE CASE 1 a __task__ modifier in the __reduction__ clause  of the __parallel__ construct is used to include the reductions of any  participating tasks, those with an __in_reduction__ clause and matching   _reduction-identifiers_  (__+__) and list items (__x__).

Note, a __taskgroup__ construct (with a __task_reduction__ clause) in not necessary to scope the explicit task reduction (as seen in the example above).  Hence, even without the implicit task reduction statement (without the C __x++__   and Fortran __x=x+1__ statements), the __task__  _reduction-modifier_   in a __reduction__ clause of the __parallel__ construct can be used to avoid having to create a __taskgroup__ construct  (and its __task_reduction__ clause) around the task generating structure.

In USE CASE 2 tasks participating in the reduction are within a worksharing region (a parallel worksharing-loop construct). Here, too, no __taskgroup__ is required, and the  _reduction-identifier_  (__+__) and list item (variable __x__) match as required.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       task_reduction.2
* type:       C
* version: omp_5.0
*/
#include <stdio.h>
int main(void){
   int N=100, M=10;
   int i, x;

// USE CASE 1  explicit-task reduction + parallel reduction clause
   x=0;
   #pragma omp parallel num_threads(M) reduction(task,+:x)
   {

     x++;                // implicit task reduction statement

     #pragma omp single
     for(i=0;i<N;i++)
       #pragma omp task in_reduction(+:x)
       x++;

   }
   printf("x=%d  =M+N\n",x);  // x= 110  =M+N


// USE CASE 2  task reduction +  worksharing reduction clause
   x=0;
   #pragma omp parallel for num_threads(M) reduction(task,+:x)
   for(i=0; i< N; i++){

      x++;

      if( i%2 == 0){
       #pragma omp task in_reduction(+:x)
       x--;
      }
   }
   printf("x=%d  =N-N/2\n",x);  // x= 50  =N-N/2

   return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       task_reduction.2
! type:       F-free
! version:    omp_5.0

program task_modifier

   integer :: N=100, M=10
   integer :: i, x

! USE CASE 1  explicit-task reduction + parallel reduction clause
   x=0
   !$omp parallel num_threads(M) reduction(task,+:x)

     x=x+1                   !! implicit task reduction statement

     !$omp single
       do i = 1,N
         !$omp task in_reduction(+:x)
           x=x+1
         !$omp end task
       end do
     !$omp end single

   !$omp end parallel
   write(*,'("x=",I0," =M+N")') x   ! x= 110 =M+N


! USE CASE 2  task reduction +  worksharing reduction clause
   x=0
   !$omp parallel do num_threads(M) reduction(task,+:x)
     do i = 1,N

        x=x+1

        if( mod(i,2) == 0) then
           !$omp task in_reduction(+:x)
             x=x-1
           !$omp end task
        endif

     end do
   write(*,'("x=",I0,"  =N-N/2")') x   ! x= 50 =N-N/2

end program

### Reduction on Combined Target Constructs

When a __reduction__ clause appears on a combined construct that combines  a __target__ construct with another construct, there is an implicit map  of the list items with a __tofrom__ map type for the __target__ construct.  Otherwise, the list items (if they are scalar variables) would be  treated as firstprivate by default in the __target__ construct, which  is unlikely to provide the intended behavior since the result of the reduction that is in the firstprivate variable would be discarded  at the end of the __target__ region.

In the following example, the use of the __reduction__ clause on __sum1__ or __sum2__ should, by default, result in an implicit __tofrom__ map for that variable. So long as neither __sum1__ nor __sum2__ were already present on the device, the mapping behavior ensures the value for __sum1__ computed in the first __target__ construct is used in the second __target__ construct.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target_reduction.1
* type: C
* version: omp_5.0
*/
#include <stdio.h>
int f(int);
int g(int);
int main()
{
   int sum1=0, sum2=0;
   int i;
   const int n = 100;

   #pragma omp target teams distribute reduction(+:sum1)
   for (int i = 0; i < n; i++) {
      sum1 += f(i);
   }

   #pragma omp target teams distribute reduction(+:sum2)
   for (int i = 0; i < n; i++) {
      sum2 += g(i) * sum1;
   }

   printf(  "sum1 = %d, sum2 = %d\n", sum1, sum2);
   //OUTPUT: sum1 = 9900, sum2 = 147015000
   return 0;
}

int f(int res){ return res*2; }
int g(int res){ return res*3; }

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: target_reduction.1
! type: F-free
! version: omp_5.0
program target_reduction_ex1
   interface
      function f(res)
             integer :: f, res
          end function
      function g(res)
             integer :: g, res
          end function
   end interface
   integer :: sum1, sum2, i
   integer, parameter :: n = 100
   sum1 = 0
   sum2 = 0
   !$omp target teams distribute reduction(+:sum1)
       do i=1,n
          sum1 = sum1 + f(i)
       end do
   !$omp target teams distribute reduction(+:sum2)
       do i=1,n
          sum2 = sum2 + g(i)*sum1
       end do
   print *, "sum1 = ", sum1, ", sum2 = ", sum2
   !!OUTPUT: sum1 =     10100 , sum2 = 153015000
end program


integer function f(res)
   integer :: res
   f = res*2
end function
integer function g(res)
   integer :: res
   g = res*3
end function

In next example,  the variables __sum1__ and __sum2__ remain on the device for the duration of the __target__ __data__ region so that it is their device copies that are updated by the reductions. Note the significance of mapping __sum1__ on the second __target__ construct; otherwise, it would be treated by default as firstprivate and the result computed for __sum1__ in the prior __target__ region may not be used. Alternatively, a __target__ __update__ construct could be used between the two __target__ constructs to update the host version of __sum1__ with the value that is in the corresponding device version after the completion of the first construct.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target_reduction.2
* type: C
* version: omp_5.0
*/
#include <stdio.h>
int f(int);
int g(int);
int main()
{
   int sum1=0, sum2=0;
   int i;
   const int n = 100;

   #pragma omp target data map(sum1,sum2)
   {
      #pragma omp target teams distribute reduction(+:sum1)
      for (int i = 0; i < n; i++) {
         sum1 += f(i);
      }

      #pragma omp target teams distribute map(sum1) reduction(+:sum2)
      for (int i = 0; i < n; i++) {
         sum2 += g(i) * sum1;
      }
   }
   printf(  "sum1 = %d, sum2 = %d\n", sum1, sum2);
   //OUTPUT: sum1 = 9900, sum2 = 147015000
   return 0;
}

int f(int res){ return res*2; }
int g(int res){ return res*3; }

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: target_reduction.2
! type: F-free
! version: omp_5.0

program target_reduction_ex2
   interface
      function f(res)
             integer :: f, res
          end function
      function g(res)
             integer :: g, res
          end function
   end interface
   integer :: sum1, sum2, i
   integer, parameter :: n = 100
   sum1 = 0
   sum2 = 0
   !$omp target data map(sum1, sum2)
       !$omp target teams distribute reduction(+:sum1)
           do i=1,n
              sum1 = sum1 + f(i)
           end do
       !$omp target teams distribute map(sum1) reduction(+:sum2)
           do i=1,n
              sum2 = sum2 + g(i)*sum1
           end do
   !$omp end target data
   print *, "sum1 = ", sum1, ", sum2 = ", sum2
   !!OUTPUT: sum1 =     10100 , sum2 = 153015000
end program


integer function f(res)
   integer :: res
   f = res*2
end function
integer function g(res)
   integer :: res
   g = res*3
end function

### Task Reduction with Target Constructs

The following examples illustrate how task reductions can apply to target tasks that result from a __target__ construct with the __in_reduction__ clause. Here, the __in_reduction__ clause specifies that the target task participates in the task reduction defined in the scope of the enclosing __taskgroup__ construct. Partial results from all tasks participating in the task reduction will be combined (in some order) into the original variable listed in the __task_reduction__ clause before exiting the __taskgroup__ region.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target_task_reduction.1
* type: C
* version: omp_5.2
*/

#include <stdio.h>
#pragma omp declare target enter(device_compute)
void device_compute(int *);
void host_compute(int *);
int main()
{
   int sum = 0;

   #pragma omp parallel masked
   #pragma omp taskgroup task_reduction(+:sum)
   {
      #pragma omp target in_reduction(+:sum) nowait
          device_compute(&sum);

      #pragma omp task in_reduction(+:sum)
          host_compute(&sum);
   }
   printf(  "sum = %d\n", sum);
   //OUTPUT: sum = 2
   return 0;
}

void device_compute(int *sum){ *sum = 1; }
void   host_compute(int *sum){ *sum = 1; }

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: target_task_reduction.1
! type: F-free
! version: omp_5.2

program target_task_reduction_ex1
   interface
      subroutine device_compute(res)
      !$omp declare target enter(device_compute)
        integer :: res
      end subroutine device_compute
      subroutine host_compute(res)
        integer :: res
      end subroutine host_compute
   end interface
   integer :: sum
   sum = 0
   !$omp parallel masked
      !$omp taskgroup task_reduction(+:sum)
         !$omp target in_reduction(+:sum) nowait
            call device_compute(sum)
         !$omp end target
         !$omp task in_reduction(+:sum)
            call host_compute(sum)
         !$omp end task
      !$omp end taskgroup
   !$omp end parallel masked
   print *, "sum = ", sum
   !!OUTPUT: sum = 2
end program

subroutine device_compute(sum)
   integer :: sum
   sum = 1
end subroutine
subroutine host_compute(sum)
   integer :: sum
   sum = 1
end subroutine

In the next pair of examples, the task reduction is defined by a __reduction__ clause with the __task__ modifier, rather than a __task_reduction__ clause on a __taskgroup__ construct. Again, the partial results from the participating tasks will be combined in some order into the original reduction variable, __sum__.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target_task_reduction.2a
* type: C
* version: omp_5.2
*/
#include <stdio.h>
#pragma omp declare target enter(device_compute)
extern void device_compute(int *);
extern void host_compute(int *);
int main()
{
   int sum = 0;

   #pragma omp parallel sections reduction(task, +:sum)
   {
      #pragma omp section
          {
             #pragma omp target in_reduction(+:sum)
             device_compute(&sum);
          }
      #pragma omp section
          {
             host_compute(&sum);
          }
   }
   printf(  "sum = %d\n", sum);
   //OUTPUT: sum = 2
   return 0;
}

void device_compute(int *sum){ *sum = 1; }
void   host_compute(int *sum){ *sum = 1; }

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: target_task_reduction.2a
! type: F-free
! version: omp_5.2

program target_task_reduction_ex2
   interface
      subroutine device_compute(res)
      !$omp declare target enter(device_compute)
        integer :: res
      end subroutine device_compute
      subroutine host_compute(res)
        integer :: res
      end subroutine host_compute
   end interface
   integer :: sum
   sum = 0
   !$omp parallel sections reduction(task,+:sum)
      !$omp section
         !$omp target in_reduction(+:sum) nowait
           call device_compute(sum)
         !$omp end target
      !$omp section
         call host_compute(sum)
   !$omp end parallel sections
   print *, "sum = ", sum
   !!OUTPUT: sum = 2
end program

subroutine device_compute(sum)
   integer :: sum
   sum = 1
end subroutine
subroutine host_compute(sum)
   integer :: sum
   sum = 1
end subroutine

Next, the __task__ modifier is again used to define a task reduction over participating tasks. This time, the participating tasks are a target task resulting from a __target__ construct with the __in_reduction__ clause, and the implicit task (executing on the primary thread) that calls __host_compute__. As before, the partial results from these participating tasks are combined in some order into the original reduction variable.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target_task_reduction.2b
* type: C
* version: omp_5.2
*/
#include <stdio.h>
#pragma omp declare target enter(device_compute)
extern void device_compute(int *);
extern void host_compute(int *);
int main()
{
   int sum = 0;

   #pragma omp parallel masked reduction(task, +:sum)
   {
       #pragma omp target in_reduction(+:sum) nowait
       device_compute(&sum);

       host_compute(&sum);
   }
   printf(  "sum = %d\n", sum);
   //OUTPUT: sum = 2
   return 0;
}

void device_compute(int *sum){ *sum = 1; }
void   host_compute(int *sum){ *sum = 1; }

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: target_task_reduction.2b
! type: F-free
! version: omp_5.2

program target_task_reduction_ex2b
   interface
      subroutine device_compute(res)
      !$omp declare target enter(device_compute)
        integer :: res
      end subroutine device_compute
      subroutine host_compute(res)
        integer :: res
      end subroutine host_compute
   end interface
   integer :: sum
   sum = 0
   !$omp parallel masked reduction(task,+:sum)
         !$omp target in_reduction(+:sum) nowait
           call device_compute(sum)
         !$omp end target
         call host_compute(sum)
   !$omp end parallel masked
   print *, "sum = ", sum
   !!OUTPUT: sum = 2
end program


subroutine device_compute(sum)
   integer :: sum
   sum = 1
end subroutine
subroutine host_compute(sum)
   integer :: sum
   sum = 1
end subroutine

### Taskloop Reduction

In the OpenMP 5.0 Specification the __taskloop__ construct was extended to include the reductions.

The following two examples show how to implement a reduction over an array using taskloop reduction in two different ways. In the first example we apply the __reduction__ clause to the __taskloop__ construct. As it was explained above in the task reduction examples, a reduction over tasks is divided in two components: the scope of the reduction, which is defined by a __taskgroup__ region, and the tasks that participate in the reduction. In this example, the __reduction__ clause defines both semantics. First, it specifies that the implicit __taskgroup__ region associated with the __taskloop__ construct is the scope of the reduction, and second, it defines all tasks created by the __taskloop__ construct as participants of the reduction. About the first property, it is important to note that if we add the __nogroup__ clause to the __taskloop__ construct the code will be nonconforming, basically because we have a set of tasks that participate in a reduction that has not been defined.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       taskloop_reduction.1
* type:       C
* version:    omp_5.0
*/
#include <stdio.h>

int array_sum(int n, int *v) {
    int i;
    int res = 0;

    #pragma omp taskloop reduction(+: res)
    for(i = 0; i < n; ++i)
        res += v[i];

    return res;
}

int main(int argc, char *argv[]) {
    int n = 10;
    int v[10] = {1,2,3,4,5,6,7,8,9,10};

    #pragma omp parallel
    #pragma omp single
    {
        int res = array_sum(n, v);
        printf("The result is %d\n", res);
    }
    return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: taskloop_reduction.1
! type: F-free
! version:    omp_5.0
function array_sum(n, v) result(res)
    implicit none
    integer :: n, v(n), res
    integer :: i

    res = 0
    !$omp taskloop reduction(+: res)
    do i=1, n
        res = res + v(i)
    end do
    !$omp end taskloop

end function array_sum

program main
    implicit none
    integer :: n, v(10), res
    integer :: i

    integer, external :: array_sum

    n = 10
    do i=1, n
        v(i) = i
    end do

    !$omp parallel
    !$omp single
    res = array_sum(n, v)
    print *, "The result is", res
    !$omp end single
    !$omp end parallel
end program main

The second example computes exactly the same value as in the preceding  _taskloop_reduction.1_  code section, but in a very different way. First, in the  _array_sum_  function a __taskgroup__ region is created  that defines the scope of a new reduction using the __task_reduction__ clause. After that, a task and also the tasks generated by a taskloop participate in  that reduction by using the __in_reduction__ clause on the __task__ and __taskloop__ constructs, respectively.  Note that the __nogroup__ clause was added to the __taskloop__ construct. This is allowed because what is expressed with the __in_reduction__ clause is different from what is expressed with the __reduction__ clause. In one case the generated tasks are specified to participate in a previously  declared reduction (__in_reduction__ clause) whereas in the other case creation of a new reduction is specified and also all tasks generated  by the taskloop will participate on it.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       taskloop_reduction.2
* type:       C
* version:    omp_5.0
*/
#include <stdio.h>

int array_sum(int n, int *v) {
    int i;
    int res = 0;

    #pragma omp taskgroup task_reduction(+: res)
    {
        if (n > 0) {
            #pragma omp task in_reduction(+: res)
            res = res + v[0];

            #pragma omp taskloop in_reduction(+: res) nogroup
            for(i = 1; i < n; ++i)
                res += v[i];
        }
    }

    return res;
}

int main(int argc, char *argv[]) {
    int n = 10;
    int v[10] = {1,2,3,4,5,6,7,8,9,10};

    #pragma omp parallel
    #pragma omp single
    {
        int res = array_sum(n, v);
        printf("The result is %d\n", res);
    }
    return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: taskloop_reduction.2
! type: F-free
! version:    omp_5.0
function array_sum(n, v) result(res)
    implicit none
    integer :: n, v(n), res
    integer :: i

    res = 0
    !$omp taskgroup task_reduction(+: res)
    if (n > 0) then
        !$omp task in_reduction(+: res)
        res = res + v(1)
        !$omp end task

        !$omp taskloop in_reduction(+: res) nogroup
        do i=2, n
            res = res + v(i)
        end do
        !$omp end taskloop
    endif
    !$omp end taskgroup

end function array_sum

program main
    implicit none
    integer :: n, v(10), res
    integer :: i

    integer, external :: array_sum

    n = 10
    do i=1, n
        v(i) = i
    end do

    !$omp parallel
    !$omp single
    res = array_sum(n, v)
    print *, "The result is", res
    !$omp end single
    !$omp end parallel
end program main

In the OpenMP 5.0 Specification, __reduction__ clauses for the __taskloop simd__ construct were also added.

The examples below compare reductions for the __taskloop__ and the __taskloop__ __simd__ constructs. These examples illustrate the use of __reduction__ clauses within  "stand-alone" __taskloop__ constructs, and the use of __in_reduction__ clauses for tasks of taskloops to participate with other reductions within the scope of a parallel region.

**taskloop reductions:**

In the  _taskloop reductions_  section of the example below,   _taskloop 1_  uses the __reduction__ clause  in a __taskloop__ construct for a sum reduction, accumulated in  _asum_ .  The behavior is as though a __taskgroup__ construct encloses the  taskloop region with a __task_reduction__ clause, and each taskloop task has an __in_reduction__ clause with the specifications  of the __reduction__ clause. At the end of the taskloop region  _asum_  contains the result of the reduction.

The next taskloop,  _taskloop 2_ , illustrates the use of the  __in_reduction__ clause to participate in a previously defined reduction scope of a __parallel__ construct.

The task reductions of  _task 2_  and  _taskloop 2_  are combined across the __taskloop__ construct and the single __task__ construct, as specified in the __reduction(task,__ __+:asum)__ clause of the __parallel__ construct. At the end of the parallel region  _asum_  contains the combined result of all reductions.

**taskloop simd reductions:**

Reductions for the __taskloop__ __simd__ construct are shown in the second half of the code. Since each component construct, __taskloop__ and __simd__,  can accept a reduction-type clause, the __taskloop__ __simd__ construct is a composite construct, and the specific application of the reduction clause is defined within the __taskloop__ __simd__ construct section of the OpenMP 5.0 Specification. The code below illustrates use cases for these reductions.

In the  _taskloop simd reduction_  section of the example below,  _taskloop simd 3_  uses the __reduction__ clause  in a __taskloop__ __simd__ construct for a sum reduction within a loop. For this case a __reduction__ clause is used, as one would use  for a __simd__ construct. The SIMD reductions of each task are combined, and the results of these tasks are further  combined just as in the __taskloop__ construct with the __reduction__ clause for  _taskloop 1_ . At the end of the taskloop region  _asum_  contains the combined result of all reductions.

If a __taskloop__ __simd__ construct is to participate in a previously defined  reduction scope, the reduction participation should be specified with a __in_reduction__ clause, as shown in the __parallel__ region enclosing  _task 4_  and  _taskloop simd 4_  code sections.

Here the __taskloop__ __simd__ construct's  __in_reduction__ clause specifies participation of the construct's tasks as  a task reduction within the scope of the parallel region.   That is, the results of each task of the __taskloop__ construct component  contribute to the reduction in a broader level, just as in  _parallel reduction a_  code section above. Also, each __simd__-component construct occurs as if it has a __reduction__ clause, and the SIMD results of each task are combined as though to form a single result for each task (that participates in the __in_reduction__ clause). At the end of the parallel region  _asum_  contains the combined result of all reductions.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: taskloop_simd_reduction.1
* type: C
* version: omp_5.1
*/
#include <stdio.h>
#define N 100

int main(){
   int i, a[N], asum=0;

   for(i=0;i<N;i++) a[i]=i;

   // taskloop reductions

   #pragma omp parallel masked
   #pragma omp taskloop reduction(+:asum) // taskloop 1
      for(i=0;i<N;i++){ asum += a[i]; }


   #pragma omp parallel reduction(task, +:asum) // parallel reduction a
   {
      #pragma omp masked
      #pragma omp task            in_reduction(+:asum) // task 2
         for(i=0;i<N;i++){ asum += a[i]; }

      #pragma omp masked taskloop in_reduction(+:asum) // taskloop 2
         for(i=0;i<N;i++){ asum += a[i]; }
   }

   // taskloop simd reductions

   #pragma omp parallel masked
   #pragma omp taskloop simd reduction(+:asum) // taskloop simd 3
     for(i=0;i<N;i++){ asum += a[i]; }


   #pragma omp parallel reduction(task, +:asum) // parallel reduction b
   {
      #pragma omp masked
      #pragma omp task                 in_reduction(+:asum) // task 4
         for(i=0;i<N;i++){ asum += a[i]; }

      #pragma omp masked taskloop simd in_reduction(+:asum) // taskloop
         for(i=0;i<N;i++){ asum += a[i]; }                  // simd 4

  }

  printf("asum=%d \n",asum); // output: asum=29700
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: taskloop_simd_reduction.1
! type: F-free
! version: omp_5.1
program main

  use omp_lib
  integer, parameter ::  N=100
  integer            :: i, a(N), asum=0

  a = [( i, i=1,N )]    !! initialize

!! taskloop reductions

  !$omp parallel masked
  !$omp taskloop reduction(+:asum)                  !! taskloop 1
    do i=1,N;  asum = asum + a(i);  enddo
  !$omp end taskloop
  !$omp end parallel masked


  !$omp parallel reduction(task, +:asum)            !! parallel reduction a

     !$omp masked
     !$omp task            in_reduction(+:asum)     !! task 2
       do i=1,N;  asum = asum + a(i);  enddo
     !$omp end task
     !$omp end masked

     !$omp masked taskloop in_reduction(+:asum)     !! taskloop 2
       do i=1,N;  asum = asum + a(i);  enddo
     !$omp end masked taskloop

  !$omp end parallel

!! taskloop simd reductions

  !$omp parallel masked
  !$omp taskloop simd reduction(+:asum)             !! taskloop simd 3
    do i=1,N;  asum = asum + a(i);  enddo
  !$omp end taskloop simd
  !$omp end parallel masked


  !$omp parallel reduction(task, +:asum)            !! parallel reduction b

    !$omp masked
    !$omp task                 in_reduction(+:asum) !! task 4
       do i=1,N;  asum = asum + a(i);  enddo
    !$omp end task
    !$omp end masked

    !$omp masked taskloop simd in_reduction(+:asum) !! taskloop simd 4
       do i=1,N;  asum = asum + a(i);  enddo
    !$omp end masked taskloop simd

  !$omp end parallel

  print*,"asum=",asum   !! output: asum=30300

end program

### Reduction with the __scope__ Construct

The following example illustrates the use of the __scope__ construct  to perform a reduction in a __parallel__ region. The case is useful for  producing a reduction and accessing reduction variables inside a __parallel__ region  without using a worksharing-loop construct.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: scope_reduction.1
* type: C++
* version: omp_5.1
*/
#include <stdio.h>
void do_work(int n, float a[], float &s)
{
   float loc_s = 0.0f;        // local sum
   static int nthrs;
   #pragma omp for
      for (int i = 0; i < n; i++)
         loc_s += a[i];
   #pragma omp single
   {
      s = 0.0f;               // total sum
      nthrs = 0;
   }
   #pragma omp scope reduction(+:s,nthrs)
   {
      s += loc_s;
      nthrs++;
   }
   #pragma omp masked
      printf("total sum = %f, nthrs = %d\n", s, nthrs);
}

float work(int n, float a[])
{
   float s;
   #pragma omp parallel
   {
      do_work(n, a, s);
   }
   return s;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: scope_reduction.1
! type: F-free
! version: omp_5.1
subroutine do_work(n, a, s)
   implicit none
   integer n, i
   real a(*), s, loc_s
   integer, save :: nthrs

   loc_s = 0.0                ! local sum
   !$omp do
      do i = 1, n
         loc_s = loc_s + a(i)
      end do
   !$omp single
      s = 0.0                 ! total sum
      nthrs = 0
   !$omp end single
   !$omp scope reduction(+:s,nthrs)
      s = s + loc_s
      nthrs = nthrs + 1
   !$omp end scope
   !$omp masked
      print *, "total sum = ", s, ", nthrs = ", nthrs
   !$omp end masked
end subroutine

function work(n, a) result(s)
   implicit none
   integer n
   real a(*), s

   !$omp parallel
      call do_work(n, a, s)
   !$omp end parallel
end function

### User-Defined Reduction

The __declare__ __reduction__ directive can be used to specify  user-defined reductions (UDR) for user data types.

In the following example, __declare__ __reduction__ directives are used to define  _min_  and  _max_  operations for the  _point_  data structure for computing the rectangle that encloses a set of 2-D points.

Each __declare__ __reduction__ directive defines new reduction identifiers,  _min_  and  _max_ , to be used in a __reduction__ clause. The next item in the declaration list is the data type ( _struct_   _point_ ) used in the reduction, followed by the combiner, here the functions  _minproc_  and  _maxproc_  perform the min and max operations, respectively, on the user data (of type  _struct_   _point_ ). In the function argument list are two special OpenMP variable identifiers, __omp_in__ and __omp_out__, that denote the two values to be combined in the "real" function; the __omp_out__ identifier indicates which one is to hold the result.

The initializer of the __declare__ __reduction__ directive specifies the initial value for the private variable of each implicit task. The __omp_priv__ identifier is used to denote the private variable.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.1
* type: C
* version: omp_4.0
*/
#include <stdio.h>
#include <limits.h>

struct point {
  int x;
  int y;
};

void minproc ( struct point *out, struct point *in )
{
  if ( in->x < out->x ) out->x = in->x;
  if ( in->y < out->y ) out->y = in->y;
}

void maxproc ( struct point *out, struct point *in )
{
  if ( in->x > out->x ) out->x = in->x;
  if ( in->y > out->y ) out->y = in->y;
}

#pragma omp declare reduction(min : struct point : \
        minproc(&omp_out, &omp_in)) \
 initializer( omp_priv = { INT_MAX, INT_MAX } )

#pragma omp declare reduction(max : struct point : \
        maxproc(&omp_out, &omp_in)) \
 initializer( omp_priv = { 0, 0 } )

void find_enclosing_rectangle ( int n, struct point points[] )
{
  struct point minp = { INT_MAX, INT_MAX }, maxp = {0,0};
  int i;

#pragma omp parallel for reduction(min:minp) reduction(max:maxp)
  for ( i = 0; i < n; i++ ) {
     minproc(&minp, &points[i]);
     maxproc(&maxp, &points[i]);
  }
  printf("min = (%d, %d)\n", minp.x, minp.y);
  printf("max = (%d, %d)\n", maxp.x, maxp.y);
}

The following example shows the corresponding code in Fortran.  The __declare__ __reduction__ directives are specified as part of  the declaration in subroutine  _find_enclosing_rectangle_  and  the procedures that perform the min and max operations are specified as subprograms.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: udr.1
! type: F-free
! version: omp_4.0
module data_type

  type :: point
    integer :: x
    integer :: y
  end type

end module data_type

subroutine find_enclosing_rectangle ( n, points )
  use data_type
  implicit none
  integer :: n
  type(point) :: points(*)

  !$omp declare reduction(min : point : minproc(omp_out, omp_in)) &
  !$omp&  initializer( omp_priv = point( HUGE(0), HUGE(0) ) )

  !$omp declare reduction(max : point : maxproc(omp_out, omp_in)) &
  !$omp&  initializer( omp_priv = point( 0, 0 ) )

  type(point) :: minp = point( HUGE(0), HUGE(0) ), maxp = point( 0, 0 )
  integer :: i

  !$omp parallel do reduction(min:minp) reduction(max:maxp)
  do i = 1, n
     call minproc(minp, points(i))
     call maxproc(maxp, points(i))
  end do
  print *, "min = (", minp%x, minp%y, ")"
  print *, "max = (", maxp%x, maxp%y, ")"

 contains
  subroutine minproc ( out, in )
    implicit none
    type(point), intent(inout) :: out
    type(point), intent(in) :: in

    out%x = min( out%x, in%x )
    out%y = min( out%y, in%y )
  end subroutine minproc

  subroutine maxproc ( out, in )
    implicit none
    type(point), intent(inout) :: out
    type(point), intent(in) :: in

    out%x = max( out%x, in%x )
    out%y = max( out%y, in%y )
  end subroutine maxproc

end subroutine

The following example shows the same computation as  _udr.1_  but it illustrates that you can craft complex expressions in the user-defined reduction declaration. In this case, instead of calling the  _minproc_  and  _maxproc_  functions we inline the code in a single expression.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.2
* type: C
* version: omp_4.0
*/
#include <stdio.h>
#include <limits.h>

struct point {
  int x;
  int y;
};

#pragma omp declare reduction(min : struct point : \
        omp_out.x = omp_in.x > omp_out.x  ? omp_out.x : omp_in.x, \
        omp_out.y = omp_in.y > omp_out.y  ? omp_out.y : omp_in.y ) \
        initializer( omp_priv = { INT_MAX, INT_MAX } )

#pragma omp declare reduction(max : struct point : \
        omp_out.x = omp_in.x < omp_out.x  ? omp_out.x : omp_in.x,  \
        omp_out.y = omp_in.y < omp_out.y  ? omp_out.y : omp_in.y ) \
        initializer( omp_priv = { 0, 0 } )

void find_enclosing_rectangle ( int n, struct point points[] )
{
  struct point minp = { INT_MAX, INT_MAX }, maxp = {0,0};
  int i;

#pragma omp parallel for reduction(min:minp) reduction(max:maxp)
  for ( i = 0; i < n; i++ ) {
    if ( points[i].x < minp.x ) minp.x = points[i].x;
    if ( points[i].y < minp.y ) minp.y = points[i].y;
    if ( points[i].x > maxp.x ) maxp.x = points[i].x;
    if ( points[i].y > maxp.y ) maxp.y = points[i].y;
  }
  printf("min = (%d, %d)\n", minp.x, minp.y);
  printf("max = (%d, %d)\n", maxp.x, maxp.y);
}

The corresponding code of the same example in Fortran is very similar except that the assignment expression in the __declare__ __reduction__ directive can only be used for a single variable, in this case through a type structure constructor  _point( ... )_ .

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: udr.2
! type: F-free
! version: omp_4.0
module data_type

  type :: point
    integer :: x
    integer :: y
  end type

end module data_type

subroutine find_enclosing_rectangle ( n, points )
  use data_type
  implicit none
  integer :: n
  type(point) :: points(*)

  !$omp declare reduction( min : point :  &
  !$omp&   omp_out = point(min( omp_out%x, omp_in%x ), &
  !$omp&                   min( omp_out%y, omp_in%y )) ) &
  !$omp&   initializer( omp_priv = point( HUGE(0), HUGE(0) ) )

  !$omp declare reduction( max : point :  &
  !$omp&   omp_out = point(max( omp_out%x, omp_in%x ), &
  !$omp&                   max( omp_out%y, omp_in%y )) ) &
  !$omp&   initializer( omp_priv = point( 0, 0 ) )

  type(point) :: minp = point( HUGE(0), HUGE(0) ), maxp = point( 0, 0 )
  integer :: i

  !$omp parallel do reduction(min: minp) reduction(max: maxp)
  do i = 1, n
     minp%x = min(minp%x, points(i)%x)
     minp%y = min(minp%y, points(i)%y)
     maxp%x = max(maxp%x, points(i)%x)
     maxp%y = max(maxp%y, points(i)%y)
  end do
  print *, "min = (", minp%x, minp%y, ")"
  print *, "max = (", maxp%x, maxp%y, ")"

end subroutine

The following example shows the use of special variables in arguments for combiner (__omp_in__ and __omp_out__) and initializer (__omp_priv__ and __omp_orig__) routines.  This example returns the maximum value of an array and the corresponding index value. The __declare__ __reduction__ directive specifies a user-defined reduction operation  _maxloc_  for data type  _struct_   _mx_s_ . The function  _mx_combine_  is the combiner and the function  _mx_init_  is the initializer.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.3
* type: C
* version: omp_4.0
*/

#include <stdio.h>
#define N 100

struct mx_s {
   float value;
   int index;
};

/* prototype functions for combiner and initializer in
   the declare reduction */
void mx_combine(struct mx_s *out, struct mx_s *in);
void mx_init(struct mx_s *priv, struct mx_s *orig);

#pragma omp declare reduction(maxloc: struct mx_s: \
        mx_combine(&omp_out, &omp_in)) \
        initializer(mx_init(&omp_priv, &omp_orig))

void mx_combine(struct mx_s *out, struct mx_s *in)
{
   if ( out->value < in->value ) {
      out->value = in->value;
      out->index = in->index;
   }
}

void mx_init(struct mx_s *priv, struct mx_s *orig)
{
   priv->value = orig->value;
   priv->index = orig->index;
}

int main(void)
{
   struct mx_s mx;
   float val[N], d;
   int i, count = N;

   for (i = 0; i < count; i++) {
      d = (N*0.8f - i);
      val[i] = N * N - d * d;
   }

   mx.value = val[0];
   mx.index = 0;
   #pragma omp parallel for reduction(maxloc: mx)
   for (i = 1; i < count; i++) {
      if (mx.value < val[i])
      {
         mx.value = val[i];
         mx.index = i;
      }
   }

   printf("max value = %g, index = %d\n", mx.value, mx.index);
   /* prints 10000, 80 */

   return 0;
}

Below is the corresponding Fortran version of the above example.  The __declare__ __reduction__ directive specifies the user-defined operation  _maxloc_  for user-derived type  _mx_s_ .  The combiner  _mx_combine_  and the initializer  _mx_init_  are specified as subprograms.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: udr.3
! type: F-free
! version: omp_4.0
program max_loc
   implicit none

   type :: mx_s
      real value
      integer index
   end type

   !$omp declare reduction(maxloc: mx_s: &
   !$omp&        mx_combine(omp_out, omp_in)) &
   !$omp&        initializer(mx_init(omp_priv, omp_orig))

   integer, parameter :: N = 100
   type(mx_s) :: mx
   real :: val(N), d
   integer :: i, count

   count = N
   do i = 1, count
      d = N*0.8 - i + 1
      val(i) = N * N - d * d
   enddo

   mx%value = val(1)
   mx%index = 1
   !$omp parallel do reduction(maxloc: mx)
   do i = 2, count
      if (mx%value < val(i)) then
         mx%value = val(i)
         mx%index = i
      endif
   enddo

   print *, 'max value = ', mx%value, ' index = ', mx%index
   ! prints 10000, 81

 contains

 subroutine mx_combine(out, in)
   implicit none
   type(mx_s), intent(inout) :: out
   type(mx_s), intent(in) :: in

   if ( out%value < in%value ) then
      out%value = in%value
      out%index = in%index
   endif
 end subroutine mx_combine

 subroutine mx_init(priv, orig)
   implicit none
   type(mx_s), intent(out) :: priv
   type(mx_s), intent(in) :: orig

   priv%value = orig%value
   priv%index = orig%index
 end subroutine mx_init

end program

The following example explains a few details of the user-defined reduction  in Fortran through modules. The __declare__ __reduction__ directive is declared in a module ( _data_red_ ).  The reduction-identifier  _.add._  is a user-defined operator that is to allow accessibility in the scope that performs the reduction operation. The user-defined operator  _.add._  and the subroutine  _dt_init_  specified in the __initializer__ clause are defined in the same subprogram.

The reduction operation (that is, the __reduction__ clause) is in the main program. The reduction identifier  _.add._  is accessible by use association. Since  _.add._  is a user-defined operator, the explicit interface should also be accessible by use association in the current program unit. Since the __declare__ __reduction__ associated to this __reduction__ clause has the __initializer__ clause, the subroutine specified on the clause must be accessible in the current scoping unit.  In this case, the subroutine  _dt_init_  is accessible by use association.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: udr.4
! type: F-free
! version: omp_4.0
module data_red
! Declare data type.
  type dt
    real :: r1
    real :: r2
  end type

! Declare the user-defined operator .add.
  interface operator(.add.)
    module procedure addc
  end interface

! Declare the user-defined reduction operator .add.
!$omp declare reduction(.add.:dt:omp_out=omp_out.add.omp_in) &
!$omp& initializer(dt_init(omp_priv))

 contains
! Declare the initialization routine.
  subroutine dt_init(u)
    type(dt) :: u
    u%r1 = 0.0
    u%r2 = 0.0
  end subroutine

! Declare the specific procedure for the .add. operator.
  function addc(x1, x2) result(xresult)
    type(dt), intent(in) :: x1, x2
    type(dt) :: xresult
    xresult%r1 = x1%r1 + x2%r2
    xresult%r2 = x1%r2 + x2%r1
  end function

end module data_red

program main
  use data_red, only : dt, dt_init, operator(.add.)

  type(dt) :: xdt1, xdt2
  integer :: i

  xdt1 = dt(1.0,2.0)
  xdt2 = dt(2.0,3.0)

! The reduction operation
!$omp parallel do reduction(.add.: xdt1)
  do i = 1, 10
    xdt1 = xdt1 .add. xdt2
  end do
!$omp end parallel do

  print *, xdt1

end program

The following example uses user-defined reductions to declare a plus (+) reduction for a C++ class. As the __declare__ __reduction__ directive is inside the context of the  _V_  class the expressions in the __declare__ __reduction__ directive are resolved in the context of the class. Also, note that the __initializer__ clause uses a copy constructor to initialize the private variables of the reduction and it uses as parameter to its original variable by using the special variable __omp_orig__.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.5
* type: C++
* version: omp_4.0
*/
class V {
   float *p;
   int n;

public:
   V( int _n )     : n(_n)  { p = new float[n]; }
   V( const V& m ) : n(m.n) { p = new float[n]; }
   ~V() { delete[] p; }

   V& operator+= ( const V& );

   #pragma omp declare reduction( + : V : omp_out += omp_in ) \
           initializer(omp_priv(omp_orig))
};

The following examples shows how user-defined reductions can be defined for some STL containers. The first __declare__ __reduction__ defines the plus (+) operation for  _std::vector<int>_  by making use of the  _std::transform_  algorithm. The second and third define the merge (or concatenation) operation for  _std::vector<int>_  and  _std::list<int>_ .  It shows how the user-defined reduction operation can be applied to specific data types of an STL.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: udr.6
* type: C++
* version: omp_4.0
*/
#include <algorithm>
#include <list>
#include <vector>

#pragma omp declare reduction( + : std::vector<int> : \
     std::transform (omp_out.begin(), omp_out.end(),  \
        omp_in.begin(), omp_in.end(),std::plus<int>()))

#pragma omp declare reduction( merge : std::vector<int> : \
     omp_out.insert(omp_out.end(), omp_in.begin(), omp_in.end()))

#pragma omp declare reduction( merge : std::list<int> : \
     omp_out.merge(omp_in))

## __scan__ Directive

The following examples illustrate how to parallelize a loop that saves  the **prefix sum** of a reduction. This is accomplished by using  the __inscan__ modifier in the __reduction__ clause for the input  variable of the scan, and specifying with a __scan__ directive whether  the storage statement includes or excludes the scan input of the present  iteration ( **k** ).

Basically, the __inscan__ modifier connects a loop and/or SIMD reduction to  the scan operation, and a __scan__ construct with an __inclusive__ or  __exclusive__ clause specifies whether the "scan phase'' (lexical block  before and after the directive, respectively) is to use an  _inclusive_  or   _exclusive_  scan value for the list item ( **x** ).

The first example uses the  _inclusive_  scan operation on a composite loop-SIMD construct. The __scan__ directive separates the reduction  statement on variable  **x**  from the use of  **x**  (saving to array  **b** ). The order of the statements in this example indicates that value  **a[k]**  ( **a(k)**  in Fortran) is included in the computation of  the prefix sum  **b[k]**  ( **b(k)**  in Fortran) for iteration  **k** .

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       scan.1
* type:       C
* version: omp_5.0
*/
#include <stdio.h>
#define N 100

int main(void)
{
   int a[N], b[N];
   int x = 0;

   // initialization
   for (int k = 0; k < N; k++)
      a[k] = k + 1;

   // a[k] is included in the computation of producing results in b[k]
   #pragma omp parallel for simd reduction(inscan,+: x)
   for (int k = 0; k < N; k++) {
      x += a[k];
      #pragma omp scan inclusive(x)
      b[k] = x;
   }

   printf("x = %d, b[0:3] = %d %d %d\n", x, b[0], b[1], b[2]);
   //           5050,        1  3  6

   return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       scan.1
! type:       F-free
! version: omp_5.0
program inclusive_scan
   implicit none
   integer, parameter :: n = 100
   integer a(n), b(n)
   integer x, k

   ! initialization
   x = 0
   do k = 1, n
      a(k) = k
   end do

   ! a(k) is included in the computation of producing results in b(k)
   !$omp parallel do simd reduction(inscan,+: x)
   do k = 1, n
      x = x + a(k)
      !$omp scan inclusive(x)
      b(k) = x
   end do

   print *,'x =', x, ', b(1:3) =', b(1:3)
   !           5050,            1  3  6

end program

The second example uses the  _exclusive_  scan operation on a composite loop-SIMD construct. The __scan__ directive separates the use of  **x**   (saving to array  **b** ) from the reduction statement on variable  **x** . The order of the statements in this example indicates that value  **a[k]**  ( **a(k)**  in Fortran) is excluded from the computation  of the prefix sum  **b[k]**  ( **b(k)**  in Fortran) for iteration  **k** .

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       scan.2
* type:       C
* version: omp_5.0
*/
#include <stdio.h>
#define N 100

int main(void)
{
   int a[N], b[N];
   int x = 0;

   // initialization
   for (int k = 0; k < N; k++)
      a[k] = k + 1;

   // a[k] is not included in the computation of producing
   // results in b[k]
   #pragma omp parallel for simd reduction(inscan,+: x)
   for (int k = 0; k < N; k++) {
      b[k] = x;
      #pragma omp scan exclusive(x)
      x += a[k];
   }

   printf("x = %d, b[0:3] = %d %d %d\n", x, b[0], b[1], b[2]);
   //           5050,        0  1  3

   return 0;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name:       scan.2
! type:       F-free
! version: omp_5.0
program exclusive_scan
   implicit none
   integer, parameter :: n = 100
   integer a(n), b(n)
   integer x, k

   ! initialization
   x = 0
   do k = 1, n
      a(k) = k
   end do

   ! a(k) is not included in the computation of producing results in b(k)
   !$omp parallel do simd reduction(inscan,+: x)
   do k = 1, n
      b(k) = x
      !$omp scan exclusive(x)
      x = x + a(k)
   end do

   print *,'x =', x, ', b(1:3) =', b(1:3)
   !           5050,            0  1  3

end program

## __copyin__ Clause

The __copyin__ clause is used to initialize threadprivate data upon entry  to a __parallel__ region. The value of the threadprivate variable in the primary thread is copied to the threadprivate variable of each other team member.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: copyin.1
* type: C
*/
#include <stdlib.h>

float* work;
int size;
float tol;

#pragma omp threadprivate(work,size,tol)

void build()
{
  int i;
  work = (float*)malloc( sizeof(float)*size );
  for( i = 0; i < size; ++i ) work[i] = tol;
}

void copyin_example( float t, int n )
{
  tol = t;
  size = n;
  #pragma omp parallel copyin(tol,size)
  {
    build();
  }
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: copyin.1
! type: F-fixed
      MODULE M
        REAL, POINTER, SAVE :: WORK(:)
        INTEGER :: SIZE
        REAL :: TOL
!$OMP   THREADPRIVATE(WORK,SIZE,TOL)
      END MODULE M

      SUBROUTINE COPYIN_EXAMPLE( T, N )
        USE M
        REAL :: T
        INTEGER :: N
        TOL = T
        SIZE = N
!$OMP   PARALLEL COPYIN(TOL,SIZE)
        CALL BUILD
!$OMP   END PARALLEL
      END SUBROUTINE COPYIN_EXAMPLE

      SUBROUTINE BUILD
        USE M
        ALLOCATE(WORK(SIZE))
        WORK = TOL
      END SUBROUTINE BUILD

## __copyprivate__ Clause

The __copyprivate__ clause can be used to broadcast values acquired by a single  thread directly to all instances of the private variables in the other threads.  In this example, if the routine is called from the sequential part, its behavior  is not affected by the presence of the directives. If it is called from a __parallel__  region, then the actual arguments with which __a__ and __b__ are associated  must be private.

The thread that executes the structured block associated with the __single__   construct broadcasts the values of the private variables __a__, __b__,  __x__, and  __y__ from its implicit task's data environment to the data environments  of the other implicit tasks in the thread team. The broadcast completes before  any of the threads have left the barrier at the end of the construct.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: copyprivate.1
* type: C
*/
#include <stdio.h>
float x, y;
#pragma omp threadprivate(x, y)

void init(float a, float b ) {
    #pragma omp single copyprivate(a,b,x,y)
    {
        scanf("%f %f %f %f", &a, &b, &x, &y);
    }
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: copyprivate.1
! type: F-fixed
      SUBROUTINE INIT(A,B)
      REAL A, B
        COMMON /XY/ X,Y
!$OMP   THREADPRIVATE (/XY/)

!$OMP   SINGLE
          READ (11) A,B,X,Y
!$OMP   END SINGLE COPYPRIVATE (A,B,/XY/)

      END SUBROUTINE INIT

In this example, assume that the input must be performed by the primary thread.  Since the __masked__ construct does not support the __copyprivate__ clause,  it cannot broadcast the input value that is read. However, __copyprivate__  is used to broadcast an address where the input value is stored.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: copyprivate.2
* type: C
* version:    omp_5.1
*/
#include <stdio.h>
#include <stdlib.h>

float read_next( ) {
  float * tmp;
  float return_val;

  #pragma omp single copyprivate(tmp)
  {
    tmp = (float *) malloc(sizeof(float));
  }  /* copies the pointer only */


  #pragma omp masked
  {
    scanf("%f", tmp);
  }

  #pragma omp barrier
  return_val = *tmp;
  #pragma omp barrier

  #pragma omp single nowait
  {
    free(tmp);
  }

  return return_val;
}

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: copyprivate.2
! type: F-fixed
! version:    omp_5.1
        REAL FUNCTION READ_NEXT()
        REAL, POINTER :: TMP

!$OMP   SINGLE
          ALLOCATE (TMP)
!$OMP   END SINGLE COPYPRIVATE (TMP)  ! copies the pointer only

!$OMP   MASKED
          READ (11) TMP
!$OMP   END MASKED

!$OMP   BARRIER
          READ_NEXT = TMP
!$OMP   BARRIER

!$OMP   SINGLE
          DEALLOCATE (TMP)
!$OMP   END SINGLE NOWAIT
        END FUNCTION READ_NEXT

Suppose that the number of lock variables required within a __parallel__ region  cannot easily be determined prior to entering it. The __copyprivate__ clause  can be used to provide access to shared lock variables that are allocated within  that __parallel__ region.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: copyprivate.3
* type: C
*/
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

omp_lock_t *new_lock()
{
  omp_lock_t *lock_ptr;

  #pragma omp single copyprivate(lock_ptr)
  {
    lock_ptr = (omp_lock_t *) malloc(sizeof(omp_lock_t));
    omp_init_lock( lock_ptr );
  }

  return lock_ptr;
}

In [ ]:

!!%compiler: gfortran
!!%cflags: -fopenmp

! name: copyprivate.3
! type: F-fixed
      FUNCTION NEW_LOCK()
      USE OMP_LIB       ! or INCLUDE "omp_lib.h"
        INTEGER(OMP_LOCK_KIND), POINTER :: NEW_LOCK

!$OMP   SINGLE
          ALLOCATE(NEW_LOCK)
          CALL OMP_INIT_LOCK(NEW_LOCK)
!$OMP   END SINGLE COPYPRIVATE(NEW_LOCK)
      END FUNCTION NEW_LOCK

Note that the effect of the __copyprivate__ clause on a variable with the  __allocatable__ attribute is different than on a variable with the __pointer__  attribute. The value of __A__ is copied (as if by intrinsic assignment) and  the pointer __B__ is copied (as if by pointer assignment) to the corresponding  list items in the other implicit tasks belonging to the __parallel__ region.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: copyprivate.4
! type: F-fixed
      SUBROUTINE S(N)
      INTEGER N

        REAL, DIMENSION(:), ALLOCATABLE :: A
        REAL, DIMENSION(:), POINTER :: B

        ALLOCATE (A(N))
!$OMP   SINGLE
          ALLOCATE (B(N))
          READ (11) A,B
!$OMP   END SINGLE COPYPRIVATE(A,B)
        ! Variable A is private and is
        ! assigned the same value in each thread
        ! Variable B is shared

!$OMP   BARRIER
!$OMP   SINGLE
          DEALLOCATE (B)
!$OMP   END SINGLE NOWAIT
      END SUBROUTINE S

## C++ Reference in Data-Sharing Clauses

C++ reference types are allowed in data-sharing attribute clauses as of OpenMP 4.5, except for the __threadprivate__, __copyin__ and __copyprivate__ clauses.   (See the Data-Sharing Attribute Clauses Section of the 4.5 OpenMP specification.) When a variable with C++ reference type is privatized, the object the reference refers to is privatized in addition to the reference itself. The following example shows the use of reference types in data-sharing clauses in the usual way. Additionally it shows how the data-sharing of formal arguments with a C++ reference type on an orphaned task generating construct is determined implicitly. (See the Data-sharing Attribute Rules for Variables Referenced in a Construct Section of the 4.5 OpenMP specification.)

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: cpp_reference.1
* type: C++
* version: omp_4.5
*/

void task_body (int &);
void gen_task (int &x) { // on orphaned task construct reference argument
  #pragma omp task // x is implicitly determined firstprivate(x)
  task_body (x);
}
void test (int &y, int &z) {
  #pragma omp parallel private(y)
  {
    y = z + 2;
    gen_task (y); // no matter if the argument is determined private
    gen_task (z); // or shared in the enclosing context.

    y++;          // each thread has its own int object y refers to
    gen_task (y);
  }
}

## Fortran __ASSOCIATE__ Construct

The following is an invalid example of specifying an associate name on a data-sharing attribute  clause. The constraint in the Data Sharing Attribute Rules section in the OpenMP  4.0 API Specifications states that an associate name preserves the association  with the selector established at the __ASSOCIATE__ statement. The associate  name  _b_  is associated with the shared variable  _a_ . With the predetermined data-sharing  attribute rule, the associate name  _b_  is not allowed to be specified on the __private__  clause.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: associate.1
! type: F-fixed
! version: omp_4.0
      program example_broken
      real :: a, c
      associate (b => a)
!$omp parallel private(b, c)        ! invalid to privatize b
      c = 2.0*b
!$omp end parallel
      end associate
      end program

In next example, within the __parallel__ construct, the association name  _thread_id_   is associated with the private copy of  _i_ . The print statement should output the  unique thread number.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: associate.2
! type: F-fixed
! version: omp_4.0
      program example
      use omp_lib
      integer  i
!$omp parallel private(i)
      i = omp_get_thread_num()
      associate(thread_id => i)
        print *, thread_id       ! print private i value
      end associate
!$omp end parallel
      end program

The following example illustrates the effect of specifying a selector name on a data-sharing  attribute clause. The associate name  _u_  is associated with  _v_  and the variable  _v_   is specified on the __private__ clause of the __parallel__ construct.  The construct association is established prior to the __parallel__ region.  The association between  _u_  and the original  _v_  is retained (see the Data Sharing  Attribute Rules section in the OpenMP 4.0 API Specifications). Inside the __parallel__  region,  _v_  has the value of -1 and  _u_  has the value of the original  _v_ .

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: associate.3
! type: F-free
! version: omp_4.0
program example
  integer :: v
  v = 15
associate(u => v)
!$omp parallel private(v)
  v = -1
  print *, v               ! private v=-1
  print *, u               ! original v=15
!$omp end parallel
end associate
end program

The following example illustrates mapping behavior for a Fortran associate name and its selector for a __target__ construct.

For the first 3 __target__ constructs the associate name  _a_aray_  is associated with the selector  _aray_ , an array.   For the __target__ construct of code block TARGET 1 just the selector  _aray_  is used and is implicitly mapped, likewise for the associate name  _a_aray_  in the TARGET 2 block.   However, mapping an associate name and its selector is not valid for the same __target__ construct.  Hence the TARGET 3 block is non-conforming.

In TARGET 4, the  _scalr_  selector used in the __target__ region  has an implicit data-sharing attribute of firstprivate since it is a scalar. Hence, the assigned value is not returned. In TARGET 5, the associate name  _a_scalr_  is implicitly mapped and the assigned value is returned to the host (default __tofrom__ mapping behavior). In TARGET 6, the use of the associate name and its selector in the __target__ region is conforming because the scalar firstprivate behavior of the selector  and the implicit mapping of the associate name are allowed.   At the end of the __target__ region only the  associate name's value is returned to the host.  In TARGET 7, the selector and associate name appear in an explicit mapping for the same __target__ construct,  hence the code block is non-conforming.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: associate.4
! type: F-free
! version: omp_5.1
program main
  integer :: scalr, aray(3)
  scalr = -1 ; aray = -1

  associate(a_scalr=>scalr, a_aray=>aray)

 !$omp target            !! TARGET 1
    aray = [1,2,3]
  !$omp end target
  print *, a_aray, aray  !!  1 2 3   1 2 3

  !$omp target           !! TARGET 2
    a_aray = [4,5,6]
  !$omp end target
  print *, a_aray, aray  !! 4 5 6   4 5 6

!!!$omp target           !! TARGET 3
!!                       !! mapping, in this case implicit,
!!                       !! of aray AND a_aray NOT ALLOWED
!!    aray = [4,5,6]
!!  a_aray = [1,2,3]
!!!$omp end target


  !$omp target              !! TARGET 4
    scalr = 1               !! scalr is firstprivate
  !$omp end target
  print *, a_scalr, scalr   !! -1  -1

  !$omp target              !! TARGET 5
    a_scalr = 2             !! a_scalr implicitly mapped
  !$omp end target
  print *, a_scalr, scalr   !!  2   2

  !$omp target              !! TARGET 6
    scalr = 3               !!          scalr is firstprivate
    print *, a_scalr, scalr !!  2   3
    a_scalr = 4             !!          a_scalr implicitly mapped
    print *, a_scalr, scalr !!  4   3
  !$omp end target
  print *, a_scalr, scalr   !!  4   4

!!!$omp target map(a_scalr,scalr)  !! TARGET 7
                                   !! mapping, in this case explicit,
                                   !! of scalr AND a_sclar NOT ALLOWED
!!    scalr = 5
!!  a_scalr = 5
!!!$omp end target

  end associate

end program